# Setup

### Import libraries, define helper functions, load data

In [19]:
import geopandas as gpd
import pandas as pd
import folium
import branca.colormap as cm

print("Libraries imported successfully!")

Libraries imported successfully!


In [20]:
# Function to load CSV files
def load_csv(filepath, name, header_row=0, skip_rows_list=None):
    try:
        df = pd.read_csv(filepath, header=header_row, skiprows=skip_rows_list)
        print(f"{name} loaded successfully from: {filepath}")
        return df
    except FileNotFoundError:
        print(f"Error: {name} file not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading {name} data from {filepath}: {e}")
        return None

# Function to load GeoJSON/Shapefiles
def load_geo_data(filepath, name):
    try:
        gdf = gpd.read_file(filepath)
        print(f"{name} loaded successfully from: {filepath}")
        return gdf
    except FileNotFoundError:
        print(f"Error: {name} file not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error loading {name} data from {filepath}: {e}")
        return None

print("--- Helper Functions Defined ---")

--- Helper Functions Defined ---


In [21]:
# --- Load COTA Bus Stops Data ---
cota_stops_path = "../data/COTA/Stops_2023_09.shp"
cota_stops_gdf = load_geo_data(cota_stops_path, "COTA Bus Stops")

# --- Load Decennial Census 2020 DHC.P1 Data ---
decennial_dhc_p1_path = "../data/US_Census/DECENNIALDHC2020.P1-Data.csv"
decennial_dhc_p1_df = load_csv(decennial_dhc_p1_path, "Decennial DHC2020.P1", 1)

# --- Load ACS 5-Year Estimate: S0101 (Age and Sex) ---
acs_s0101_path = "../data/US_Census/ACSST5Y2023.S0101-Data.csv"
acs_s0101_df = load_csv(acs_s0101_path, "ACS S0101 (Age and Sex)", 1)

# --- Load ACS 5-Year Estimate: B08101 (Means of Transportation to Work by Age) ---
acs_b08101_path = "../data/US_Census/ACSDT5Y2023.B08101-Data.csv"
acs_b08101_df = load_csv(acs_b08101_path, "ACS B08101 (Transportation by Age)", 1)

# --- Load ACS 5-Year Estimate: B08119 (Means of Transportation to Work by Workers' Earnings) ---
acs_b08119_path = "../data/US_Census/ACSDT5Y2023.B08119-Data.csv"
acs_b08119_df = load_csv(acs_b08119_path, "ACS B08119 (Transportation by Earnings)", 1)

# --- Load ACS 5-Year Estimate: B08122 (Means of Transportation to Work by Poverty Status) ---
acs_b08122_path = "../data/US_Census/ACSDT5Y2023.B08122-Data.csv"
acs_b08122_df = load_csv(acs_b08122_path, "ACS B08122 (Transportation by Poverty)", 1)

# --- Load ACS 5-Year Estimate: B08141 (Means of Transportation to Work by Vehicles Available) ---
acs_b08141_path = "../data/US_Census/ACSDT5Y2023.B08141-Data.csv"
acs_b08141_df = load_csv(acs_b08141_path, "ACS B08141 (Vehicles Available)", 1)

# --- Load MORPC TAZ Forecast Data ---
morpc_taz_path = "../data/MORPC/TAZ20_OfficialMTP2024_Forecasts.shp"
morpc_taz_gdf = load_geo_data(morpc_taz_path, "MORPC TAZ Forecasts")

# --- Load FTA Monthly Ridership Data ---
fta_ridership_path = "../data/FTA/May2025_RawMonthlyRidership.xlsx"
try:
    fta_ridership_df = pd.read_excel(fta_ridership_path, skiprows=0)
    print(f"FTA Monthly Ridership data loaded successfully from: {fta_ridership_path}")
except FileNotFoundError:
    print(f"Error: FTA Monthly Ridership file not found at {fta_ridership_path}")
except Exception as e:
    print(f"Error loading FTA Monthly Ridership data from {fta_ridership_path}: {e}")
    
print("\n--- All Data Loading Attempts Complete ---")

COTA Bus Stops loaded successfully from: ../data/COTA/Stops_2023_09.shp
Decennial DHC2020.P1 loaded successfully from: ../data/US_Census/DECENNIALDHC2020.P1-Data.csv
ACS S0101 (Age and Sex) loaded successfully from: ../data/US_Census/ACSST5Y2023.S0101-Data.csv
ACS B08101 (Transportation by Age) loaded successfully from: ../data/US_Census/ACSDT5Y2023.B08101-Data.csv
ACS B08119 (Transportation by Earnings) loaded successfully from: ../data/US_Census/ACSDT5Y2023.B08119-Data.csv
ACS B08122 (Transportation by Poverty) loaded successfully from: ../data/US_Census/ACSDT5Y2023.B08122-Data.csv
ACS B08141 (Vehicles Available) loaded successfully from: ../data/US_Census/ACSDT5Y2023.B08141-Data.csv
MORPC TAZ Forecasts loaded successfully from: ../data/MORPC/TAZ20_OfficialMTP2024_Forecasts.shp
Error: FTA Monthly Ridership file not found at ../data/FTA/May2025_RawMonthlyRidership.xlsx

--- All Data Loading Attempts Complete ---


### Data cleaning and merging

In [22]:
# --- Clean COTA Bus Stops Data ---
print("--- Cleaning COTA Bus Stops Data ---")
if cota_stops_gdf is not None:
    # 1. Standardize All Column Names
    cota_stops_gdf = cota_stops_gdf.rename(columns={
        'StopId': 'stop_id',
        'StopAbbr': 'stop_abbr',
        'StopName': 'stop_name',
        'OnStreet': 'on_street',
        'AtStreet': 'at_street',
        'InService_': 'in_service',
        'Jurisdicti': 'jurisdiction',
        'Amenity_Ow': 'amenity_owner',
        'Lines_Serv': 'lines_served',
        
        'Shelter': 'has_shelter',
        'Lighting': 'has_lighting',
        'Garbage': 'has_garbage_can',

        # ------- Ridership Columns (Onboards) -------
        # Avg Daily Onboardings January 2023 Trimester
        'Ons_2023_0': 'onboards_2023_T1',
        # Avg Daily Onboardings May 2023 Trimester
        'Ons_2023_1': 'onboards_2023_T2',
        # Avg Daily Onboardings September 2023 Trimester
        'Ons_2023_2': 'onboards_2023_T3',
        
        # Avg Daily Onboardings January 2024 Trimester
        'Ons_2024_0': 'onboards_2024_T1',
        # Avg Daily Onboardings May 2024 Trimester
        'Ons_2024_1': 'onboards_2024_T2',
        # Avg Daily Onboardings September 2024 Trimester
        'Ons_2024_2': 'onboards_2024_T3',

        # ------- Ridership Columns (Offboards) -------
        # Avg Daily Alightings January 2023 Trimester
        'Offs_2023_': 'offboards_2023_T1',
        # Avg Daily Alightings May 2023 Trimester
        'Offs_20231': 'offboards_2023_T2',
        # Avg Daily Alightings September 2023 Trimester
        'Offs_20232': 'offboards_2023_T3',

        # Avg Daily Alightings January 2024 Trimester
        'Offs_2024_': 'offboards_2024_T1',
        # Avg Daily Alightings May 2024 Trimester
        'Offs_20241': 'offboards_2024_T2',
        # Avg Daily Alightings September 2024 Trimester
        'Offs_20242': 'offboards_2024_T3',
    })
    print("Columns renamed.")

    # Create total_onboards column.
    cota_stops_gdf["total_onboards"] = cota_stops_gdf["onboards_2023_T1"] + cota_stops_gdf["onboards_2023_T2"] + cota_stops_gdf["onboards_2023_T3"] + cota_stops_gdf["onboards_2024_T1"] + cota_stops_gdf["onboards_2024_T2"] + cota_stops_gdf["onboards_2024_T3"]
    cota_stops_gdf["total_onboards"] = cota_stops_gdf['total_onboards'].astype(float).round(1)
    
    # 2. Convert Amenity Columns to Binary
    # For 'has_shelter'
    if cota_stops_gdf['has_shelter'].dtype == 'object':
        cota_stops_gdf['has_shelter'] = cota_stops_gdf['has_shelter'].map({'Yes': 1, 'No': 0}).fillna(0).astype(int)

    # For 'has_lighting'
    if cota_stops_gdf['has_lighting'].dtype == 'object':
        cota_stops_gdf['has_lighting'] = cota_stops_gdf['has_lighting'].replace({'Solar': 'Yes', 'Other': 'Yes', 'CMAX': 'Yes'})
        cota_stops_gdf['has_lighting'] = cota_stops_gdf['has_lighting'].map({'Yes': 1, 'No': 0}).fillna(0).astype(int)

    # For 'has_garbage_can': Convert to int
    cota_stops_gdf['has_garbage_can'] = pd.to_numeric(cota_stops_gdf['has_garbage_can'], errors='coerce').fillna(0).astype(int)
    print("Amenity columns converted to 0/1.")

    # 3. Handle Missing Values
    # Fill 'at_street' with 'Unknown' for stops not at intersections
    cota_stops_gdf['at_street'] = cota_stops_gdf['at_street'].fillna('Unknown')
    
    # Fill remaining small number of missing values for categorical columns with 'Unknown'
    for col in ['jurisdiction', 'lines_served']:
        if cota_stops_gdf[col].isnull().any(): # Only process if missing values exist
            cota_stops_gdf[col] = cota_stops_gdf[col].fillna('Unknown')
    print("Missing values handled.")

    # 4. Convert StopId to object
    cota_stops_gdf['stop_id'] = cota_stops_gdf['stop_id'].astype(str)
    print("stop_id converted to string.")

    # 5. Drop Unnecessary Columns
    if 'amenity_owner' in cota_stops_gdf.columns:
        cota_stops_gdf = cota_stops_gdf.drop(columns=['amenity_owner'])
    if 'in_service' in cota_stops_gdf.columns:
        cota_stops_gdf = cota_stops_gdf.drop(columns=['in_service'])
    if 'GlobalID' in cota_stops_gdf.columns:
        cota_stops_gdf = cota_stops_gdf.drop(columns=['GlobalID'])
    print("Dropped unneccesary columns.")

    print("\n--- COTA Bus Stops Data Cleaned ---")
    print("Cleaned COTA Stops DataFrame Info (verify types and non-nulls):")
    print(cota_stops_gdf.info())
    print("\nCleaned COTA Stops Head (first few rows to verify cleanliness):")
    print(cota_stops_gdf.head())
    print("\nUnique values in final cleaned amenity columns (should be [0, 1]):")
    print("has_shelter:", cota_stops_gdf['has_shelter'].unique())
    print("has_lighting:", cota_stops_gdf['has_lighting'].unique())
    print("has_garbage_can:", cota_stops_gdf['has_garbage_can'].unique())

else:
    print("COTA Bus Stops GeoDataFrame not loaded. Cleaning skipped.")

--- Cleaning COTA Bus Stops Data ---
Columns renamed.
Amenity columns converted to 0/1.
Missing values handled.
stop_id converted to string.
Dropped unneccesary columns.

--- COTA Bus Stops Data Cleaned ---
Cleaned COTA Stops DataFrame Info (verify types and non-nulls):
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2966 entries, 0 to 2965
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   stop_id            2966 non-null   object  
 1   stop_abbr          2966 non-null   object  
 2   stop_name          2966 non-null   object  
 3   on_street          2966 non-null   object  
 4   at_street          2966 non-null   object  
 5   has_shelter        2966 non-null   int64   
 6   has_lighting       2966 non-null   int64   
 7   has_garbage_can    2966 non-null   int64   
 8   onboards_2023_T1   2966 non-null   int64   
 9   offboards_2023_T1  2966 non-null   int64   
 10  jurisdiction       29

In [23]:
# --- Clean ACS S0101 (Age and Sex) Data ---
print("--- Cleaning ACS S0101 Data ---")
if acs_s0101_df is not None:
    acs_s0101_cleaned_df = acs_s0101_df.copy() 

    # Drop the 'Unnamed' column
    if 'Unnamed: 458' in acs_s0101_cleaned_df.columns and acs_s0101_cleaned_df['Unnamed: 458'].isnull().all():
        acs_s0101_cleaned_df = acs_s0101_cleaned_df.drop(columns=['Unnamed: 458'])
        print("Dropped 'Unnamed: 458' column.")

    # Drop Margin of Error, Percent-based Estimates, and Allocation columns
    cols_to_drop_initial = [col for col in acs_s0101_cleaned_df.columns if 
                            'Margin of Error' in col or 
                            'PERCENT ALLOCATED' in col or
                            'Estimate!!Percent' in col
                           ]
    acs_s0101_cleaned_df = acs_s0101_cleaned_df.drop(columns=cols_to_drop_initial)
    print(f"Dropped {len(cols_to_drop_initial)} error/percentage/allocation columns initially.")

    # --- Direct Mapping for Column Renaming ---
    s0101_column_rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name',
        'Estimate!!Total!!Total population': 'pop_total',
        'Estimate!!Total!!Total population!!AGE!!Under 5 years': 'pop_age_under_5',
        'Estimate!!Total!!Total population!!AGE!!5 to 9 years': 'pop_age_5_to_9',
        'Estimate!!Total!!Total population!!AGE!!10 to 14 years': 'pop_age_10_to_14',
        'Estimate!!Total!!Total population!!AGE!!15 to 19 years': 'pop_age_15_to_19',
        'Estimate!!Total!!Total population!!AGE!!20 to 24 years': 'pop_age_20_to_24',
        'Estimate!!Total!!Total population!!AGE!!25 to 29 years': 'pop_age_25_to_29',
        'Estimate!!Total!!Total population!!AGE!!30 to 34 years': 'pop_age_30_to_34',
        'Estimate!!Total!!Total population!!AGE!!35 to 39 years': 'pop_age_35_to_39',
        'Estimate!!Total!!Total population!!AGE!!40 to 44 years': 'pop_age_40_to_44',
        'Estimate!!Total!!Total population!!AGE!!45 to 49 years': 'pop_age_45_to_49',
        'Estimate!!Total!!Total population!!AGE!!50 to 54 years': 'pop_age_50_to_54',
        'Estimate!!Total!!Total population!!AGE!!55 to 59 years': 'pop_age_55_to_59',
        'Estimate!!Total!!Total population!!AGE!!60 to 64 years': 'pop_age_60_to_64',
        'Estimate!!Total!!Total population!!AGE!!65 to 69 years': 'pop_age_65_to_69',
        'Estimate!!Total!!Total population!!AGE!!70 to 74 years': 'pop_age_70_to_74',
        'Estimate!!Total!!Total population!!AGE!!75 to 79 years': 'pop_age_75_to_79',
        'Estimate!!Total!!Total population!!AGE!!80 to 84 years': 'pop_age_80_to_84',
        'Estimate!!Total!!Total population!!AGE!!85 years and over': 'pop_age_85_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!5 to 14 years': 'pop_selected_age_5_to_14',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 17 years': 'pop_selected_age_15_to_17',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!Under 18 years': 'pop_selected_age_under_18',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 to 24 years': 'pop_selected_age_18_to_24',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!15 to 44 years': 'pop_selected_age_15_to_44',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!16 years and over': 'pop_selected_age_16_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over': 'pop_selected_age_18_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!21 years and over': 'pop_selected_age_21_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!60 years and over': 'pop_selected_age_60_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!62 years and over': 'pop_selected_age_62_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!65 years and over': 'pop_selected_age_65_plus',
        'Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!75 years and over': 'pop_selected_age_75_plus',
        'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Median age (years)': 'summary_ind_median_age_years',
        'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Sex ratio (males per 100 females)': 'summary_ind_sex_ratio',
        'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Age dependency ratio': 'summary_ind_age_dependency_ratio',
        'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Old-age dependency ratio': 'summary_ind_old_age_dependency_ratio',
        'Estimate!!Total!!Total population!!SUMMARY INDICATORS!!Child dependency ratio': 'summary_ind_child_dependency_ratio',
        'Estimate!!Male!!Total population': 'male_pop_total',
        'Estimate!!Female!!Total population': 'female_pop_total'
    }

    # Apply the direct renaming.
    acs_s0101_cleaned_df = acs_s0101_cleaned_df.rename(columns={k: v for k, v in s0101_column_rename_map.items() if k in acs_s0101_cleaned_df.columns})
    print("Columns renamed.")

    # Convert all relevant population columns to numeric and fill NaNs
    for col in acs_s0101_cleaned_df.columns:
        if col not in ['geo_id', 'geographic_area_name']:
            acs_s0101_cleaned_df[col] = pd.to_numeric(acs_s0101_cleaned_df[col], errors='coerce')
    print("All population columns converted to numeric.")

    # Handle any remaining NaN values - fill with 0
    acs_s0101_cleaned_df = acs_s0101_cleaned_df.fillna(0)
    print("NaN values filled with 0.")
    
    # Create aggregated age group. 5 to 14 years = 5 to 9 years + 10 to 14 years
    if 'pop_age_5_to_9' in acs_s0101_cleaned_df.columns and 'pop_age_10_to_14' in acs_s0101_cleaned_df.columns:
        acs_s0101_cleaned_df['pop_age_5_to_14'] = acs_s0101_cleaned_df['pop_age_5_to_9'] + acs_s0101_cleaned_df['pop_age_10_to_14']
        print("Created 'pop_age_5_to_14' column.")
    else:
        print("Could not create 'pop_age_5_to_14': required columns missing.")

    # Define the final list of columns we want to keep after all cleaning and aggregated age group addition.
    final_s0101_cols_to_keep = [
        'geo_id',
        'geographic_area_name',
        'pop_total',
        'pop_age_under_5',
        'pop_age_5_to_14',            # New
        'pop_age_15_to_19',
        'pop_age_20_to_24',
        'pop_age_25_to_29',
        'pop_age_30_to_34',
        'pop_age_35_to_39',
        'pop_age_40_to_44',
        'pop_age_45_to_49',
        'pop_age_50_to_54',
        'pop_age_55_to_59',
        'pop_age_60_to_64',
        'pop_age_65_to_69',
        'pop_age_70_to_74',
        'pop_age_75_to_79',
        'pop_age_80_to_84',
        'pop_age_85_plus',
        'pop_selected_age_under_18',
        'pop_selected_age_18_to_24',
        'pop_selected_age_15_to_44',
        'pop_selected_age_16_plus',
        'pop_selected_age_18_plus',
        'pop_selected_age_21_plus',
        'pop_selected_age_60_plus',
        'pop_selected_age_62_plus',
        'pop_selected_age_65_plus',
        'pop_selected_age_75_plus',
        'summary_ind_median_age_years',
        'summary_ind_sex_ratio',
        'summary_ind_age_dependency_ratio',
        'summary_ind_old_age_dependency_ratio',
        'summary_ind_child_dependency_ratio',
        'male_pop_total',
        'female_pop_total' 
    ]

    # Filter to keep only the final desired columns.
    acs_s0101_cleaned_df = acs_s0101_cleaned_df[acs_s0101_cleaned_df.columns.intersection(final_s0101_cols_to_keep)].copy()
    print(f"Filtered to {len(acs_s0101_cleaned_df.columns)} final key population columns.")

    print("\n--- ACS S0101 Data Cleaned ---")
    print("Cleaned ACS S0101 DataFrame Info:")
    print(acs_s0101_cleaned_df.info())
    print("\nCleaned ACS S0101 Head:")
    print(acs_s0101_cleaned_df.head())
    
else:
    print("ACS S0101 DataFrame not loaded. Cleaning skipped.")

--- Cleaning ACS S0101 Data ---
Dropped 'Unnamed: 458' column.
Dropped 348 error/percentage/allocation columns initially.
Columns renamed.
All population columns converted to numeric.
NaN values filled with 0.
Created 'pop_age_5_to_14' column.
Filtered to 37 final key population columns.

--- ACS S0101 Data Cleaned ---
Cleaned ACS S0101 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 37 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   geo_id                                328 non-null    object 
 1   geographic_area_name                  328 non-null    object 
 2   pop_total                             328 non-null    int64  
 3   pop_age_under_5                       328 non-null    int64  
 4   pop_age_15_to_19                      328 non-null    int64  
 5   pop_age_20_to_24                      328 non-null    int64  
 6  

C:\Users\erolz\AppData\Local\Temp\ipykernel_31160\2896744562.py:80: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  acs_s0101_cleaned_df['pop_age_5_to_14'] = acs_s0101_cleaned_df['pop_age_5_to_9'] + acs_s0101_cleaned_df['pop_age_10_to_14']


In [24]:
# --- Clean ACS B08141 (Vehicles Available) Data ---
print("\n--- Cleaning ACS B08141 Data ---")
if acs_b08141_df is not None:
    acs_b08141_cleaned_df = acs_b08141_df.copy()

    # Drop 'Unnamed' column
    cols_to_drop_unnamed = [col for col in acs_b08141_cleaned_df.columns if 'Unnamed' in col and acs_b08141_cleaned_df[col].isnull().all()]
    if cols_to_drop_unnamed:
        acs_b08141_cleaned_df = acs_b08141_cleaned_df.drop(columns=cols_to_drop_unnamed)
        print("Dropped 'Unnamed' column.")

    # Drop Margin of Error columns
    cols_to_drop_moe = [col for col in acs_b08141_cleaned_df.columns if 'Margin of Error' in col]
    acs_b08141_cleaned_df = acs_b08141_cleaned_df.drop(columns=cols_to_drop_moe)
    print("Dropped 'Margin of Error' columns.")

    # --- Direct Mapping for B08141 Column Renaming ---
    b08141_column_rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name',
        
        # Vehicle Availability for Households
        
        'Estimate!!Total:': 'households_total', 
        'Estimate!!Total:!!No vehicle available': 'households_no_vehicle',
        'Estimate!!Total:!!1 vehicle available': 'households_1_vehicle',
        'Estimate!!Total:!!2 vehicles available': 'households_2_vehicles',
        'Estimate!!Total:!!3 or more vehicles available': 'households_3_plus_vehicles',

        # Means of Transportation to Work (Universe: Workers 16 years and over)
        
        'Estimate!!Total:!!Public transportation (excluding taxicab):': 'commute_public_transit_total',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!No vehicle available': 'commute_public_transit_no_vehicle',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!1 vehicle available': 'commute_public_transit_1_vehicle',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!2 vehicles available': 'commute_public_transit_2_vehicles',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!3 or more vehicles available': 'commute_public_transit_3_plus_vehicles',
        
        'Estimate!!Total:!!Car, truck, or van - drove alone:': 'commute_drove_alone_total',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!No vehicle available': 'commute_drove_alone_no_vehicle',
        
        'Estimate!!Total:!!Walked:': 'commute_walked_total',
        'Estimate!!Total:!!Worked from home:': 'commute_worked_from_home_total',
        'Estimate!!Total:!!Taxicab, motorcycle, bicycle, or other means:': 'commute_other_means_total'
    }

    # Apply the direct renaming.
    acs_b08141_cleaned_df = acs_b08141_cleaned_df.rename(columns={k: v for k, v in b08141_column_rename_map.items() if k in acs_b08141_cleaned_df.columns})
    print("Columns renamed.")

    # --- Select Key Columns for Analysis from B08141 ---
    final_b08141_cols = [
        'geo_id',
        'geographic_area_name',
        'households_total',
        'households_no_vehicle',
        'households_1_vehicle',
        'households_2_vehicles',
        'households_3_plus_vehicles',
        'commute_public_transit_total',
        'commute_public_transit_no_vehicle',
        'commute_public_transit_1_vehicle',
        'commute_public_transit_2_vehicles',
        'commute_public_transit_3_plus_vehicles',
        'commute_drove_alone_total',
        'commute_walked_total',
        'commute_worked_from_home_total',
        'commute_other_means_total'
    ]
    acs_b08141_cleaned_df = acs_b08141_cleaned_df[acs_b08141_cleaned_df.columns.intersection(final_b08141_cols)].copy()
    print(f"Filtered to {len(acs_b08141_cleaned_df.columns)} key vehicle availability and commuting columns.")

    # Convert all numerical columns to numeric and fill NaNs with 0
    for col in acs_b08141_cleaned_df.columns:
        if col not in ['geo_id', 'geographic_area_name']:
            acs_b08141_cleaned_df[col] = pd.to_numeric(acs_b08141_cleaned_df[col], errors='coerce').fillna(0).astype(int)
    print("Numerical columns converted to int and NaNs filled with 0.")

    print("\n--- ACS B08141 Data Cleaned ---")
    print("Cleaned ACS B08141 DataFrame Info:")
    print(acs_b08141_cleaned_df.info())
    print("\nCleaned ACS B08141 Head:")
    print(acs_b08141_cleaned_df.head())

else:
    print("ACS B08141 DataFrame not loaded. Cleaning skipped.")


--- Cleaning ACS B08141 Data ---
Dropped 'Unnamed' column.
Dropped 'Margin of Error' columns.
Columns renamed.
Filtered to 16 key vehicle availability and commuting columns.
Numerical columns converted to int and NaNs filled with 0.

--- ACS B08141 Data Cleaned ---
Cleaned ACS B08141 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 16 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   geo_id                                  328 non-null    object
 1   geographic_area_name                    328 non-null    object
 2   households_total                        328 non-null    int64 
 3   households_no_vehicle                   328 non-null    int64 
 4   households_1_vehicle                    328 non-null    int64 
 5   households_2_vehicles                   328 non-null    int64 
 6   households_3_plus_vehicles              328 

In [25]:
# --- Clean ACS B08101 (Means of Transportation to Work by Age) Data ---
print("\n--- Cleaning ACS B08101 Data ---")
if acs_b08101_df is not None:
    acs_b08101_cleaned_df = acs_b08101_df.copy()

    # Drop 'Unnamed' column
    cols_to_drop_unnamed = [col for col in acs_b08101_cleaned_df.columns if 'Unnamed' in col and acs_b08101_cleaned_df[col].isnull().all()]
    if cols_to_drop_unnamed:
        acs_b08101_cleaned_df = acs_b08101_cleaned_df.drop(columns=cols_to_drop_unnamed)
        print("Dropped 'Unnamed' column.")

    # Drop Margin of Error columns
    cols_to_drop_moe = [col for col in acs_b08101_cleaned_df.columns if 'Margin of Error' in col]
    acs_b08101_cleaned_df = acs_b08101_cleaned_df.drop(columns=cols_to_drop_moe)
    print("Dropped 'Margin of Error' columns.")

    # --- Direct Mapping for B08101 Column Renaming ---
    b08101_column_rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name',
        
        # Total Workers by Age Group (Universe: Workers 16 years and over)
        'Estimate!!Total:': 'workers_total',
        'Estimate!!Total:!!16 to 19 years': 'workers_age_16_to_19',
        'Estimate!!Total:!!20 to 24 years': 'workers_age_20_to_24',
        'Estimate!!Total:!!25 to 44 years': 'workers_age_25_to_44',
        'Estimate!!Total:!!45 to 54 years': 'workers_age_45_to_54',
        'Estimate!!Total:!!55 to 59 years': 'workers_age_55_to_59',
        'Estimate!!Total:!!60 to 64 years': 'workers_age_60_to_64',
        'Estimate!!Total:!!65 years and over': 'workers_age_65_plus',

        # Commute by Public Transportation (excluding taxicab) by Age Group
        'Estimate!!Total:!!Public transportation (excluding taxicab):': 'commute_public_transit_total',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!16 to 19 years': 'commute_public_transit_age_16_to_19',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!20 to 24 years': 'commute_public_transit_age_20_to_24',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!25 to 44 years': 'commute_public_transit_age_25_to_44',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!45 to 54 years': 'commute_public_transit_age_45_to_54',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!55 to 59 years': 'commute_public_transit_age_55_to_59',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!60 to 64 years': 'commute_public_transit_age_60_to_64',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!65 years and over': 'commute_public_transit_age_65_plus',

        # Commute by Car, Truck, or Van - Drove Alone by Age Group (for comparison)
        'Estimate!!Total:!!Car, truck, or van - drove alone:': 'commute_drove_alone_total',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!16 to 19 years': 'commute_drove_alone_age_16_to_19',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!20 to 24 years': 'commute_drove_alone_age_20_to_24',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!25 to 44 years': 'commute_drove_alone_age_25_to_44',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!45 to 54 years': 'commute_drove_alone_age_45_to_54',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!55 to 59 years': 'commute_drove_alone_age_55_to_59',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!60 to 64 years': 'commute_drove_alone_age_60_to_64',
        'Estimate!!Total:!!Car, truck, or van - drove alone:!!65 years and over': 'commute_drove_alone_age_65_plus',

        # Commute by Worked from home by Age Group
        'Estimate!!Total:!!Worked from home': 'commute_worked_from_home_total',
        'Estimate!!Total:!!Worked from home!!16 to 19 years': 'commute_worked_from_home_age_16_to_19',
        'Estimate!!Total:!!Worked from home!!20 to 24 years': 'commute_worked_from_home_age_20_to_24',
        'Estimate!!Total:!!Worked from home!!25 to 44 years': 'commute_worked_from_home_age_25_to_44',
        'Estimate!!Total:!!Worked from home!!45 to 54 years': 'commute_worked_from_home_age_45_to_54',
        'Estimate!!Total:!!Worked from home!!55 to 59 years': 'commute_worked_from_home_age_55_to_59',
        'Estimate!!Total:!!Worked from home!!60 to 64 years': 'commute_worked_from_home_age_60_to_64',
        'Estimate!!Total:!!Worked from home!!65 years and over': 'commute_worked_from_home_age_65_plus',

        # Other useful overall commute modes (not by age breakdown)
        'Estimate!!Total:!!Car, truck, or van - carpooled:': 'commute_carpooled_total',
        'Estimate!!Total:!!Walked:': 'commute_walked_total',
        'Estimate!!Total:!!Taxicab, motorcycle, bicycle, or other means:': 'commute_other_means_total'
    }

    # Apply the direct renaming.
    acs_b08101_cleaned_df = acs_b08101_cleaned_df.rename(columns={k: v for k, v in b08101_column_rename_map.items() if k in acs_b08101_cleaned_df.columns})
    print("Columns renamed.")
    
    # --- Select Key Columns for Analysis from B08101 ---
    final_b08101_cols = [
        'geo_id',
        'geographic_area_name',
        'workers_total', # Overall total workers 16+
        
        # Total Workers by Age Group
        'workers_age_16_to_19',
        'workers_age_20_to_24',
        'workers_age_25_to_44',
        'workers_age_45_to_54',
        'workers_age_55_to_59',
        'workers_age_60_to_64',
        'workers_age_65_plus',

        # Commute by Public Transportation (excluding taxicab) by Age Group
        'commute_public_transit_total',
        'commute_public_transit_age_16_to_19',
        'commute_public_transit_age_20_to_24',
        'commute_public_transit_age_25_to_44',
        'commute_public_transit_age_45_to_54',
        'commute_public_transit_age_55_to_59',
        'commute_public_transit_age_60_to_64',
        'commute_public_transit_age_65_plus',

        # Other useful total commute modes
        'commute_drove_alone_total',
        'commute_carpooled_total',
        'commute_walked_total',
        'commute_worked_from_home_total',
        'commute_other_means_total'
    ]
    acs_b08101_cleaned_df = acs_b08101_cleaned_df[acs_b08101_cleaned_df.columns.intersection(final_b08101_cols)].copy()
    print(f"Filtered to {len(acs_b08101_cleaned_df.columns)} key commuting by age columns.")

    # Convert all numerical columns to numeric and fill NaNs with 0
    for col in acs_b08101_cleaned_df.columns:
        if col not in ['geo_id', 'geographic_area_name']:
            acs_b08101_cleaned_df[col] = pd.to_numeric(acs_b08101_cleaned_df[col], errors='coerce').fillna(0).astype(int)
    print("Numerical columns converted to int and NaNs filled with 0.")

    print("\n--- ACS B08101 Data Cleaned ---")
    print("Cleaned ACS B08101 DataFrame Info:")
    print(acs_b08101_cleaned_df.info())
    print("\nCleaned ACS B08101 Head:")
    print(acs_b08101_cleaned_df.head())

else:
    print("ACS B08101 DataFrame not loaded. Cleaning skipped.")


--- Cleaning ACS B08101 Data ---
Dropped 'Unnamed' column.
Dropped 'Margin of Error' columns.
Columns renamed.
Filtered to 23 key commuting by age columns.
Numerical columns converted to int and NaNs filled with 0.

--- ACS B08101 Data Cleaned ---
Cleaned ACS B08101 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   geo_id                               328 non-null    object
 1   geographic_area_name                 328 non-null    object
 2   workers_total                        328 non-null    int64 
 3   workers_age_16_to_19                 328 non-null    int64 
 4   workers_age_20_to_24                 328 non-null    int64 
 5   workers_age_25_to_44                 328 non-null    int64 
 6   workers_age_45_to_54                 328 non-null    int64 
 7   workers_age_55_to_59 

In [26]:
# --- Clean ACS B08119 (Means of Transportation to Work by Workers' Earnings) Data ---
print("\n--- Cleaning ACS B08119 Data ---")
if acs_b08119_df is not None:
    acs_b08119_cleaned_df = acs_b08119_df.copy()

    # Drop 'Unnamed' column
    cols_to_drop_unnamed = [col for col in acs_b08119_cleaned_df.columns if 'Unnamed' in col and acs_b08119_cleaned_df[col].isnull().all()]
    if cols_to_drop_unnamed:
        acs_b08119_cleaned_df = acs_b08119_cleaned_df.drop(columns=cols_to_drop_unnamed)
        print("Dropped 'Unnamed' column.")

    # Drop Margin of Error columns
    cols_to_drop_moe = [col for col in acs_b08119_cleaned_df.columns if 'Margin of Error' in col]
    acs_b08119_cleaned_df = acs_b08119_cleaned_df.drop(columns=cols_to_drop_moe)
    print("Dropped 'Margin of Error' columns.")

    # --- Direct Mapping for B08119 Column Renaming ---
    b08119_column_rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name',
        
        # Total Workers by Earnings (Universe: Workers 16 years and over with earnings)
        'Estimate!!Total:': 'workers_earnings_total', # Overall total workers with earnings
        'Estimate!!Total:!!$1 to $9,999 or loss': 'workers_earnings_1_to_9999',
        'Estimate!!Total:!!$10,000 to $14,999': 'workers_earnings_10000_to_14999',
        'Estimate!!Total:!!$15,000 to $24,999': 'workers_earnings_15000_to_24999',
        'Estimate!!Total:!!$25,000 to $34,999': 'workers_earnings_25000_to_34999',
        'Estimate!!Total:!!$35,000 to $49,999': 'workers_earnings_35000_to_49999',
        'Estimate!!Total:!!$50,000 to $64,999': 'workers_earnings_50000_to_64999',
        'Estimate!!Total:!!$65,000 to $74,999': 'workers_earnings_65000_to_74999',
        'Estimate!!Total:!!$75,000 or more': 'workers_earnings_75000_plus',

        # Commute by Public Transportation (excluding taxicab) by Earnings Group
        'Estimate!!Total:!!Public transportation (excluding taxicab):': 'commute_public_transit_total', # Total public transit commuters (all incomes)
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$1 to $9,999 or loss': 'commute_public_transit_earnings_1_to_9999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$10,000 to $14,999': 'commute_public_transit_earnings_10000_to_14999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$15,000 to $24,999': 'commute_public_transit_earnings_15000_to_24999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$25,000 to $34,999': 'commute_public_transit_earnings_25000_to_34999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$35,000 to $49,999': 'commute_public_transit_earnings_35000_to_49999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$50,000 to $64,999': 'commute_public_transit_earnings_50000_to_64999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$65,000 to $74,999': 'commute_public_transit_earnings_65000_to_74999',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!$75,000 or more': 'commute_public_transit_earnings_75000_plus',

        # Other useful total commute modes
        'Estimate!!Total:!!Car, truck, or van - drove alone:': 'commute_drove_alone_total',
        'Estimate!!Total:!!Car, truck, or van - carpooled:': 'commute_carpooled_total',
        'Estimate!!Total:!!Walked:': 'commute_walked_total',
        'Estimate!!Total:!!Worked from home:': 'commute_worked_from_home_total',
        'Estimate!!Total:!!Taxicab, motorcycle, bicycle, or other means:': 'commute_other_means_total'
    }

    # Apply the direct renaming.
    acs_b08119_cleaned_df = acs_b08119_cleaned_df.rename(columns={k: v for k, v in b08119_column_rename_map.items() if k in acs_b08119_cleaned_df.columns})
    print("Columns renamed.")

    # --- Select Key Columns for Analysis from B08119 ---
    final_b08119_cols = [
        'geo_id',
        'geographic_area_name',
        'workers_earnings_total',
        'workers_earnings_1_to_9999',
        'workers_earnings_10000_to_14999',
        'workers_earnings_15000_to_24999',
        'workers_earnings_25000_to_34999',
        'workers_earnings_35000_to_49999',
        'workers_earnings_50000_to_64999',
        'workers_earnings_65000_to_74999',
        'workers_earnings_75000_plus',
        'commute_public_transit_total',
        'commute_public_transit_earnings_1_to_9999',
        'commute_public_transit_earnings_10000_to_14999',
        'commute_public_transit_earnings_15000_to_24999',
        'commute_public_transit_earnings_25000_to_34999',
        'commute_public_transit_earnings_35000_to_49999',
        'commute_public_transit_earnings_50000_to_64999',
        'commute_public_transit_earnings_65000_to_74999',
        'commute_public_transit_earnings_75000_plus',
        'commute_drove_alone_total',
        'commute_carpooled_total',
        'commute_walked_total',
        'commute_worked_from_home_total',
        'commute_other_means_total'
    ]
    acs_b08119_cleaned_df = acs_b08119_cleaned_df[acs_b08119_cleaned_df.columns.intersection(final_b08119_cols)].copy()
    print(f"Filtered to {len(acs_b08119_cleaned_df.columns)} key earnings and commuting columns.")

    for col in acs_b08119_cleaned_df.columns:
        if col not in ['geo_id', 'geographic_area_name']:
            acs_b08119_cleaned_df[col] = pd.to_numeric(acs_b08119_cleaned_df[col], errors='coerce').fillna(0).astype(int)
    print("Numerical columns converted to int and NaNs filled with 0.")

    print("\n--- ACS B08119 Data Cleaned ---")
    print("Cleaned ACS B08119 DataFrame Info:")
    print(acs_b08119_cleaned_df.info())
    print("\nCleaned ACS B08119 Head:")
    print(acs_b08119_cleaned_df.head())

else:
    print("ACS B08119 DataFrame not loaded. Cleaning skipped.")


--- Cleaning ACS B08119 Data ---
Dropped 'Unnamed' column.
Dropped 'Margin of Error' columns.
Columns renamed.
Filtered to 25 key earnings and commuting columns.
Numerical columns converted to int and NaNs filled with 0.

--- ACS B08119 Data Cleaned ---
Cleaned ACS B08119 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 25 columns):
 #   Column                                          Non-Null Count  Dtype 
---  ------                                          --------------  ----- 
 0   geo_id                                          328 non-null    object
 1   geographic_area_name                            328 non-null    object
 2   workers_earnings_total                          328 non-null    int64 
 3   workers_earnings_1_to_9999                      328 non-null    int64 
 4   workers_earnings_10000_to_14999                 328 non-null    int64 
 5   workers_earnings_15000_to_24999                 328 non-null    int6

In [27]:
# --- Clean ACS B08122 (Means of Transportation to Work by Poverty Status) Data ---
print("\n--- Cleaning ACS B08122 Data ---")
if acs_b08122_df is not None:
    acs_b08122_cleaned_df = acs_b08122_df.copy()

    # Drop 'Unnamed' column
    cols_to_drop_unnamed = [col for col in acs_b08122_cleaned_df.columns if 'Unnamed' in col and acs_b08122_cleaned_df[col].isnull().all()]
    if cols_to_drop_unnamed:
        acs_b08122_cleaned_df = acs_b08122_cleaned_df.drop(columns=cols_to_drop_unnamed)
        print("Dropped 'Unnamed' column.")

    # Drop Margin of Error columns
    cols_to_drop_moe = [col for col in acs_b08122_cleaned_df.columns if 'Margin of Error' in col]
    acs_b08122_cleaned_df = acs_b08122_cleaned_df.drop(columns=cols_to_drop_moe)
    print("Dropped 'Margin of Error' columns.")

    # --- Direct Mapping for B08122 Column Renaming ---
    b08122_column_rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name',
        
        # Total Workers by Poverty Level (Universe: Workers 16 years and over)
        'Estimate!!Total:': 'workers_poverty_total', # Overall total workers
        'Estimate!!Total:!!Below 100 percent of the poverty level': 'workers_poverty_below_100',
        'Estimate!!Total:!!100 to 149 percent of the poverty level': 'workers_poverty_100_to_149',
        'Estimate!!Total:!!At or above 150 percent of the poverty level': 'workers_poverty_above_150',

        # Commute by Public Transportation (excluding taxicab) by Poverty Level
        'Estimate!!Total:!!Public transportation (excluding taxicab):': 'commute_public_transit_total', # Total public transit commuters (all poverty levels)
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!Below 100 percent of the poverty level': 'commute_public_transit_poverty_below_100',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!100 to 149 percent of the poverty level': 'commute_public_transit_poverty_100_to_149',
        'Estimate!!Total:!!Public transportation (excluding taxicab):!!At or above 150 percent of the poverty level': 'commute_public_transit_poverty_above_150',

        # Other useful total commute modes
        'Estimate!!Total:!!Car, truck, or van - drove alone:': 'commute_drove_alone_total',
        'Estimate!!Total:!!Car, truck, or van - carpooled:': 'commute_carpooled_total',
        'Estimate!!Total:!!Walked:': 'commute_walked_total',
        'Estimate!!Total:!!Worked from home:': 'commute_worked_from_home_total',
        'Estimate!!Total:!!Taxicab, motorcycle, bicycle, or other means:': 'commute_other_means_total'
    }

    # Apply the direct renaming.
    acs_b08122_cleaned_df = acs_b08122_cleaned_df.rename(columns={k: v for k, v in b08122_column_rename_map.items() if k in acs_b08122_cleaned_df.columns})
    print("Columns renamed.")

    # --- Select Key Columns for Analysis from B08122 ---
    final_b08122_cols = [
        'geo_id',
        'geographic_area_name',
        'workers_poverty_total',
        'workers_poverty_below_100',
        'workers_poverty_100_to_149',
        'workers_poverty_above_150',
        'commute_public_transit_total',
        'commute_public_transit_poverty_below_100',
        'commute_public_transit_poverty_100_to_149',
        'commute_public_transit_poverty_above_150',
        'commute_drove_alone_total',
        'commute_carpooled_total',
        'commute_walked_total',
        'commute_worked_from_home_total',
        'commute_other_means_total'
    ]
    acs_b08122_cleaned_df = acs_b08122_cleaned_df[acs_b08122_cleaned_df.columns.intersection(final_b08122_cols)].copy()
    print(f"Filtered to {len(acs_b08122_cleaned_df.columns)} key poverty and commuting columns.")

    # Convert all numerical columns to numeric and fill NaNs with 0
    for col in acs_b08122_cleaned_df.columns:
        if col not in ['geo_id', 'geographic_area_name']:
            acs_b08122_cleaned_df[col] = pd.to_numeric(acs_b08122_cleaned_df[col], errors='coerce').fillna(0).astype(int)
    print("Numerical columns converted to int and NaNs filled with 0.")

    print("\n--- ACS B08122 Data Cleaned ---")
    print("Cleaned ACS B08122 DataFrame Info:")
    print(acs_b08122_cleaned_df.info())
    print("\nCleaned ACS B08122 Head:")
    print(acs_b08122_cleaned_df.head())

else:
    print("ACS B08122 DataFrame not loaded. Cleaning skipped.")


--- Cleaning ACS B08122 Data ---
Dropped 'Unnamed' column.
Dropped 'Margin of Error' columns.
Columns renamed.
Filtered to 14 key poverty and commuting columns.
Numerical columns converted to int and NaNs filled with 0.

--- ACS B08122 Data Cleaned ---
Cleaned ACS B08122 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 14 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   geo_id                                     328 non-null    object
 1   geographic_area_name                       328 non-null    object
 2   workers_poverty_total                      328 non-null    int64 
 3   workers_poverty_below_100                  328 non-null    int64 
 4   workers_poverty_100_to_149                 328 non-null    int64 
 5   workers_poverty_above_150                  328 non-null    int64 
 6   commute_drove_alone_total        

In [28]:
# --- Clean Decennial DHC2020.P1 Data (Comprehensive) ---
print("\n--- Cleaning Decennial DHC2020.P1 Data ---")
if decennial_dhc_p1_df is not None:
    decennial_dhc_p1_cleaned_df = decennial_dhc_p1_df.copy()

    # Drop 'Unnamed' column
    cols_to_drop_unnamed = [col for col in decennial_dhc_p1_cleaned_df.columns if 'Unnamed' in col and decennial_dhc_p1_cleaned_df[col].isnull().all()]
    if cols_to_drop_unnamed:
        decennial_dhc_p1_cleaned_df = decennial_dhc_p1_cleaned_df.drop(columns=cols_to_drop_unnamed)
        print(f"Dropped 'Unnamed' column.")

    # 1. Standardize column names
    rename_map = {
        'Geography': 'geo_id',
        'Geographic Area Name': 'geographic_area_name'
    }
    # Rename this column manually. Some whitespace issue (?) prevents it from being able to be renamed using the map
    decennial_dhc_p1_cleaned_df.columns.values[2] = 'total_pop_2020'
    
    # Apply the rename map
    decennial_dhc_p1_cleaned_df = decennial_dhc_p1_cleaned_df.rename(columns={k: v for k, v in rename_map.items() if k in decennial_dhc_p1_cleaned_df.columns})
    print("Columns renamed.")

    # 2. Select only the necessary columns (final filter)
    final_dhc_cols = ['geo_id', 'geographic_area_name', 'total_pop_2020']
    decennial_dhc_p1_cleaned_df = decennial_dhc_p1_cleaned_df[decennial_dhc_p1_cleaned_df.columns.intersection(final_dhc_cols)].copy()
    
    # 3. Convert population column to numeric and fill NaNs
    if 'total_pop_2020' in decennial_dhc_p1_cleaned_df.columns:
        decennial_dhc_p1_cleaned_df['total_pop_2020'] = pd.to_numeric(decennial_dhc_p1_cleaned_df['total_pop_2020'], errors='coerce').fillna(0).astype(int)
        print("Population column converted to numeric and NaNs filled.")
    else:
        print("Warning: 'total_pop_2020' column not found in final DataFrame. This means it was either not identified or filtered out.")

    print("\n--- Decennial DHC2020.P1 Data Cleaned ---")


--- Cleaning Decennial DHC2020.P1 Data ---
Dropped 'Unnamed' column.
Columns renamed.
Population column converted to numeric and NaNs filled.

--- Decennial DHC2020.P1 Data Cleaned ---


In [29]:
# --- Clean MORPC TAZ Forecast Data ---
print("\n--- Cleaning MORPC TAZ Forecast Data ---")
morpc_taz_cleaned_gdf = morpc_taz_gdf.copy()

# Rename Columns
taz_column_rename_map = {
    'TraffDist': 'traffic_district',
    'county': 'county_name',
    'TAZ2020': 'taz_id', # Primary TAZ identifier
    'Centroid20': 'taz_centroid_id',
    'TAZ2020_N': 'taz_numeric_id', # Numeric version of TAZ ID
    'MPO': 'mpo_code', # Metropolitan Planning Organization code
    
    # Population Forecasts by Year
    'POP2021': 'pop_2021',
    'U182021': 'pop_under_18_2021',
    'A18642021': 'pop_18_to_64_2021',
    'O652021': 'pop_over_65_2021',
    'POPGQ2021': 'pop_group_quarters_2021',
    'POP2025': 'pop_2025',
    'U182025': 'pop_under_18_2025',
    'A18642025': 'pop_18_to_64_2025',
    'O652025': 'pop_over_65_2025',
    'POPGQ2025': 'pop_group_quarters_2025',
    'POP2030': 'pop_2030',
    'U182030': 'pop_under_18_2030',
    'A18642030': 'pop_18_to_64_2030',
    'O652030': 'pop_over_65_2030',
    'POPGQ2030': 'pop_group_quarters_2030',
    'POP2035': 'pop_2035',
    'U182035': 'pop_under_18_2035',
    'A18642035': 'pop_18_to_64_2035',
    'O652035': 'pop_over_65_2035',
    'POPGQ2035': 'pop_group_quarters_2035',
    'POP2040': 'pop_2040',
    'U182040': 'pop_under_18_2040',
    'A18642040': 'pop_18_to_64_2040',
    'O652040': 'pop_over_65_2040',
    'POPGQ2040': 'pop_group_quarters_2040',
    'POP2045': 'pop_2045',
    'U182045': 'pop_under_18_2045',
    'A18642045': 'pop_18_to_64_2045',
    'O652045': 'pop_over_65_2045',
    'POPGQ2045': 'pop_group_quarters_2045',
    'POP2050': 'pop_2050',
    'U182050': 'pop_under_18_2050',
    'A18642050': 'pop_18_to_64_2050',
    'O652050': 'pop_over_65_2050',
    'POPGQ2050': 'pop_group_quarters_2050',

    # Job Forecasts by Year and Type
    'WRK2021': 'jobs_total_2021',
    'OFF2021': 'jobs_office_2021',
    'RTG2021': 'jobs_retail_goods_2021',
    'RTS2021': 'jobs_retail_service_2021',
    'IND2021': 'jobs_industrial_2021',
    'OTH2021': 'jobs_other_2021',
    'WRK2025': 'jobs_total_2025',
    'OFF2025': 'jobs_office_2025',
    'RTG2025': 'jobs_retail_goods_2025',
    'RTS2025': 'jobs_retail_service_2025',
    'IND2025': 'jobs_industrial_2025',
    'OTH2025': 'jobs_other_2025',
    'WRK2030': 'jobs_total_2030',
    'OFF2030': 'jobs_office_2030',
    'RTG2030': 'jobs_retail_goods_2030',
    'RTS2030': 'jobs_retail_service_2030',
    'IND2030': 'jobs_industrial_2030',
    'OTH2030': 'jobs_other_2030',
    'WRK2035': 'jobs_total_2035',
    'OFF2035': 'jobs_office_2035',
    'RTG2035': 'jobs_retail_goods_2035',
    'RTS2035': 'jobs_retail_service_2035',
    'IND2035': 'jobs_industrial_2035',
    'OTH2035': 'jobs_other_2035',
    'WRK2040': 'jobs_total_2040',
    'OFF2040': 'jobs_office_2040',
    'RTG2040': 'jobs_retail_goods_2040',
    'RTS2040': 'jobs_retail_service_2040',
    'IND2040': 'jobs_industrial_2040',
    'OTH2040': 'jobs_other_2040',
    'WRK2045': 'jobs_total_2045',
    'OFF2045': 'jobs_office_2045',
    'RTG2045': 'jobs_retail_goods_2045',
    'RTS2045': 'jobs_retail_service_2045',
    'IND2045': 'jobs_industrial_2045',
    'OTH2045': 'jobs_other_2045',
    'WRK2050': 'jobs_total_2050',
    'OFF2050': 'jobs_office_2050',
    'RTG2050': 'jobs_retail_goods_2050',
    'RTS2050': 'jobs_retail_service_2050',
    'IND2050': 'jobs_industrial_2050',
    'OTH2050': 'jobs_other_2050',
    'Jobs2021': 'jobs_total_raw_2021',
    'Jobs2050': 'jobs_total_raw_2050', 

    # Household Data
    'HHINC2021': 'household_income_2021',
    'TOTHH2021': 'households_total_2021',
    'HHINC2025': 'household_income_2025',
    'TOTHH2025': 'households_total_2025',
    'HHINC2030': 'household_income_2030',
    'TOTHH2030': 'households_total_2030',
    'HHINC2035': 'household_income_2035',
    'TOTHH2035': 'households_total_2035',
    'HHINC2040': 'household_income_2040',
    'TOTHH2040': 'households_total_2040',
    'HHINC2045': 'household_income_2045',
    'TOTHH2045': 'households_total_2045',
    'HHINC2050': 'household_income_2050',
    'TOTHH2050': 'households_total_2050',

    # School Enrollment Data
    'K8ENR2021': 'enroll_k8_2021',
    'HSENR2021': 'enroll_hs_2021',
    'ENRUNV21': 'enroll_univ_2021',
    'K8ENR2025': 'enroll_k8_2025',
    'HSENR25': 'enroll_hs_2025',
    'ENRUNV25': 'enroll_univ_2025',
    'K8ENR2030': 'enroll_k8_2030',
    'HSENR2030': 'enroll_hs_2030',
    'ENRUNV30': 'enroll_univ_2030',
    'K8ENR2035': 'enroll_k8_2035',
    'HSENR2035': 'enroll_hs_2035',
    'ENRUNV35': 'enroll_univ_2035',
    'K8ENR2040': 'enroll_k8_2040',
    'HSENR2040': 'enroll_hs_2040',
    'ENRUNV40': 'enroll_univ_2040',
    'K8ENR2045': 'enroll_k8_2045',
    'HSENR2045': 'enroll_hs_2045',
    'ENRUNV45': 'enroll_univ_2045',
    'K8ENR2050': 'enroll_k8_2050',
    'HSENR2050': 'enroll_hs_2050',
    'ENRUNV50': 'enroll_univ_2050',

    # Growth and Density Indicators
    'Job2150Gr': 'job_growth_21_50_category',
    'Job2150GD': 'job_growth_by_acre_21_50_category',
    'Pop2150Gr': 'pop_growth_21_50_total',
    'Jobs21n': 'jobs_2021_number',
    'Jobs30n': 'jobs_2030_number',
    'Jobs40n': 'jobs_2040_number',
    'Jobs50n': 'jobs_2050_number',
    'Job21dp': 'jobs_2021_density_category',
    'Job30dp': 'jobs_2030_density_category',
    'Job40dp': 'jobs_2040_density_category',
    'Job50dp': 'jobs_2050_density_category',
    'TAZacre': 'taz_acres',
    'Shape__Area': 'shape_area',
    'Shape__Length': 'shape_length'
}

# Apply the direct renaming.
morpc_taz_cleaned_gdf = morpc_taz_cleaned_gdf.rename(columns={k: v for k, v in taz_column_rename_map.items() if k in morpc_taz_cleaned_gdf.columns})
print("Columns renamed.")

# Only keep Franklin County rows.
for row in morpc_taz_cleaned_gdf:
    morpc_taz_cleaned_gdf = morpc_taz_cleaned_gdf[morpc_taz_cleaned_gdf['county_name'].str.contains("FRA", na=False)]

# Select Key Columns for Analysis
final_taz_cols_to_keep = [
    'taz_id', # Primary identifier
    'county_name', # For Franklin County filtering
    'pop_2021', 'pop_2025', 'pop_2030', 'pop_2035', 'pop_2040', 'pop_2045', 'pop_2050',
    'pop_under_18_2021', 'pop_18_to_64_2021', 'pop_over_65_2021', # Age breakdowns for 2021
    'pop_under_18_2030', 'pop_18_to_64_2030', 'pop_over_65_2030', # Age breakdowns for 2030
    'pop_under_18_2050', 'pop_18_to_64_2050', 'pop_over_65_2050', # Age breakdowns for 2050
    'pop_group_quarters_2021', 'pop_group_quarters_2030', 'pop_group_quarters_2050', # Group quarters (dorm-like)

    'jobs_total_2021', 'jobs_total_2025', 'jobs_total_2030', 'jobs_total_2035', 'jobs_total_2040', 'jobs_total_2045', 'jobs_total_2050',
    'jobs_office_2021', 'jobs_retail_goods_2021', 'jobs_retail_service_2021', 'jobs_industrial_2021', 'jobs_other_2021', # 2021 Job types
    'jobs_office_2050', 'jobs_retail_goods_2050', 'jobs_retail_service_2050', 'jobs_industrial_2050', 'jobs_other_2050', # 2050 Job types

    'households_total_2021', 'households_total_2030', 'households_total_2050',
    
    'enroll_k8_2021', 'enroll_hs_2021', 'enroll_univ_2021', # School enrollment 2021
    'enroll_univ_2030', 'enroll_univ_2050', # University enrollment forecasts

    'pop_growth_21_50_total', # Integer growth
    'job_growth_21_50_category', # Categorical growth
    'job_growth_by_acre_21_50_category', # Categorical growth by acre

    'jobs_2021_density_category', # Categorical density
    'jobs_2030_density_category',
    'jobs_2040_density_category',
    'jobs_2050_density_category',

    'taz_acres', # For calculating densities later if needed
    'geometry' # For GeoDataFrame
]

morpc_taz_cleaned_gdf = morpc_taz_cleaned_gdf[morpc_taz_cleaned_gdf.columns.intersection(final_taz_cols_to_keep)].copy()
print(f"Filtered to {len(morpc_taz_cleaned_gdf.columns)} key TAZ forecast columns.")

# Convert all relevant numerical columns to numeric (coerce errors to NaN, then fill with 0)
for col in morpc_taz_cleaned_gdf.columns:
    if col not in ['taz_id', 'county_name', 'job_growth_21_50_category', 'job_growth_by_acre_21_50_category',
                    'jobs_2021_density_category', 'jobs_2030_density_category', 'jobs_2040_density_category', 'jobs_2050_density_category',
                    'geometry']: # Exclude string/object and geometry columns
        morpc_taz_cleaned_gdf[col] = pd.to_numeric(morpc_taz_cleaned_gdf[col], errors='coerce').fillna(0)
        if morpc_taz_cleaned_gdf[col].dtype == 'float64':
            if (morpc_taz_cleaned_gdf[col] == morpc_taz_cleaned_gdf[col].astype(int)).all():
                morpc_taz_cleaned_gdf[col] = morpc_taz_cleaned_gdf[col].astype(int)
print("Numerical columns converted and NaNs filled.")

print("\n--- MORPC TAZ Forecast Data Cleaned ---")
print("Cleaned MORPC TAZ GeoDataFrame Info:")
print(morpc_taz_cleaned_gdf.info())
print("\nCleaned MORPC TAZ Head:")
print(morpc_taz_cleaned_gdf.head())


--- Cleaning MORPC TAZ Forecast Data ---
Columns renamed.
Filtered to 55 key TAZ forecast columns.
Numerical columns converted and NaNs filled.

--- MORPC TAZ Forecast Data Cleaned ---
Cleaned MORPC TAZ GeoDataFrame Info:
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1166 entries, 979 to 2175
Data columns (total 55 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   county_name                        1166 non-null   object  
 1   taz_id                             1166 non-null   object  
 2   pop_2021                           1166 non-null   int64   
 3   pop_under_18_2021                  1166 non-null   int64   
 4   pop_18_to_64_2021                  1166 non-null   int64   
 5   pop_over_65_2021                   1166 non-null   int64   
 6   pop_group_quarters_2021            1166 non-null   int64   
 7   jobs_total_2021                    1166 non-null   int64   
 8   households

In [30]:
# --- Standardize Geographic IDs for Merging ---
print("\n--- Standardizing Geographic IDs across Census DataFrames ---")

# List of cleaned Census DataFrames
census_dfs = {
    "decennial_dhc_p1": decennial_dhc_p1_cleaned_df,
    "acs_s0101": acs_s0101_cleaned_df,
    "acs_b08141": acs_b08141_cleaned_df,
    "acs_b08101": acs_b08101_cleaned_df,
    "acs_b08119": acs_b08119_cleaned_df,
    "acs_b08122": acs_b08122_cleaned_df,
}

for name, df in census_dfs.items():
    if df is not None and 'geo_id' in df.columns:
        # The 'geo_id' from data.census.gov is typically '1400000US39049XXXXXX' for tracts.
        # Standardize it to just the FIPS code for the tract (e.g., '39049XXXXXX')
        df['geo_id'] = df['geo_id'].astype(str).str.replace('1400000US', '')
        print(f"Standardized 'geo_id' in {name}_cleaned_df. Example: {df['geo_id'].iloc[0]}")
    else:
        print(f"{name}_cleaned_df not loaded or missing 'geo_id'.")

print("\n--- Geo ID Standardization Complete ---")


--- Standardizing Geographic IDs across Census DataFrames ---
Standardized 'geo_id' in decennial_dhc_p1_cleaned_df. Example: 39049000110
Standardized 'geo_id' in acs_s0101_cleaned_df. Example: 39049000110
Standardized 'geo_id' in acs_b08141_cleaned_df. Example: 39049000110
Standardized 'geo_id' in acs_b08101_cleaned_df. Example: 39049000110
Standardized 'geo_id' in acs_b08119_cleaned_df. Example: 39049000110
Standardized 'geo_id' in acs_b08122_cleaned_df. Example: 39049000110

--- Geo ID Standardization Complete ---


In [31]:
# --- Merge All Cleaned Census DataFrames ---
print("\n--- Merging Cleaned Census DataFrames (Handling duplicate columns) ---")

# Start with Decennial DHC2020.P1 as the base DataFrame
merged_census_df = decennial_dhc_p1_cleaned_df.copy()
print(f"Starting merge with decennial_dhc_p1_cleaned_df. Shape: {merged_census_df.shape}")

# List of other Census DataFrames to merge
other_census_dfs = {
    "acs_s0101": acs_s0101_cleaned_df,
    "acs_b08141": acs_b08141_cleaned_df,
    "acs_b08101": acs_b08101_cleaned_df,
    "acs_b08119": acs_b08119_cleaned_df,
    "acs_b08122": acs_b08122_cleaned_df,
}

# Define the common 'commute_total' columns that might cause duplicates
# These columns we only keep from our chosen source (B08141)
common_commute_totals = [
    'commute_drove_alone_total',
    'commute_carpooled_total',
    'commute_public_transit_total',
    'commute_walked_total',
    'commute_worked_from_home_total',
    'commute_other_means_total'
]

# Perform merges
for name, df_to_merge in other_census_dfs.items():
    if df_to_merge is not None and 'geo_id' in df_to_merge.columns:
        df_to_merge_clean = df_to_merge.copy()

        # 1. Drop 'geographic_area_name' from df_to_merge_clean if present in both
        if 'geographic_area_name' in df_to_merge_clean.columns and 'geographic_area_name' in merged_census_df.columns:
            df_to_merge_clean = df_to_merge_clean.drop(columns=['geographic_area_name'])
        
        # 2. Drop common columns from df_to_merge_clean
        cols_to_drop_from_right = [col for col in common_commute_totals if col in merged_census_df.columns and col in df_to_merge_clean.columns]
        if cols_to_drop_from_right:
            df_to_merge_clean = df_to_merge_clean.drop(columns=cols_to_drop_from_right)
            print(f"  - Dropped duplicate general commute totals from {name}_cleaned_df: {cols_to_drop_from_right}")

        # Perform merge
        initial_cols = merged_census_df.shape[1]
        merged_census_df = pd.merge(merged_census_df, df_to_merge_clean, on='geo_id', how='left')
        final_cols = merged_census_df.shape[1]
        
        print(f"Merged {name}_cleaned_df. Added {final_cols - initial_cols} columns. New shape: {merged_census_df.shape}")
    else:
        print(f"Warning: {name}_cleaned_df not loaded or missing 'geo_id'. Skipping merge for this DataFrame.")

print("\n--- Census DataFrames Merged ---")
print("Merged Census DataFrame Info:")
print(merged_census_df.info())
print("\nMerged Census DataFrame Head:")
print(merged_census_df.head())
print("\nMissing values in Merged Census DataFrame after merge:")
print(merged_census_df.isnull().sum())


--- Merging Cleaned Census DataFrames (Handling duplicate columns) ---
Starting merge with decennial_dhc_p1_cleaned_df. Shape: (328, 3)
Merged acs_s0101_cleaned_df. Added 35 columns. New shape: (328, 38)
Merged acs_b08141_cleaned_df. Added 14 columns. New shape: (328, 52)
  - Dropped duplicate general commute totals from acs_b08101_cleaned_df: ['commute_drove_alone_total', 'commute_public_transit_total', 'commute_walked_total', 'commute_worked_from_home_total', 'commute_other_means_total']
Merged acs_b08101_cleaned_df. Added 16 columns. New shape: (328, 68)
  - Dropped duplicate general commute totals from acs_b08119_cleaned_df: ['commute_drove_alone_total', 'commute_carpooled_total', 'commute_public_transit_total', 'commute_walked_total', 'commute_worked_from_home_total', 'commute_other_means_total']
Merged acs_b08119_cleaned_df. Added 17 columns. New shape: (328, 85)
  - Dropped duplicate general commute totals from acs_b08122_cleaned_df: ['commute_drove_alone_total', 'commute_carpo

In [32]:
print("\n--- Reprojecting COTA Bus Stops Data ---")
if cota_stops_gdf is not None:
    print(f"Original COTA Stops CRS: {cota_stops_gdf.crs}")
    
    # Reproject to EPSG:3735 (matching MORPC TAZ data)
    cota_stops_gdf = cota_stops_gdf.to_crs(epsg=3735)
    
    print(f"New COTA Stops CRS: {cota_stops_gdf.crs}")
    print("\nReprojection complete for COTA Bus Stops.")
else:
    print("COTA Bus Stops GeoDataFrame not loaded. Reprojection skipped.")


--- Reprojecting COTA Bus Stops Data ---
Original COTA Stops CRS: EPSG:4326
New COTA Stops CRS: EPSG:3735

Reprojection complete for COTA Bus Stops.


In [33]:
print("\n--- Creating census_tracts_gdf ---")
shp_file = "../data/US_Census/Tract_Shapefiles/tl_2020_39049_tract20.shp"
census_tracts_gdf = None

try:
    census_tracts_gdf = gpd.read_file(shp_file)
    print(f"Successfully loaded shapefile from: {shp_file}")
    print(f"Initial Census Tract GeoDataFrame loaded. Total features: {len(census_tracts_gdf)}")
    
    print(f"Original CRS: {census_tracts_gdf.crs}")

    # --- Reproject Census Tracts to EPSG:3735 ---
    if census_tracts_gdf.crs != 'EPSG:3735':
        print(f"\nReprojecting Census Tract Boundaries from {census_tracts_gdf.crs} to EPSG:3735...")
        census_tracts_gdf = census_tracts_gdf.to_crs(epsg=3735)
        print(f"New Census Tract Boundaries CRS: {census_tracts_gdf.crs}")
    else:
        print("\nCensus Tract Boundaries are already in EPSG:3735. No reprojection needed.")

    # --- Standardize and Select Final Columns ---
    cols_to_keep_from_tract_shapefile = ['GEOID20', 'NAME20', 'geometry']
    census_tracts_gdf = census_tracts_gdf[census_tracts_gdf.columns.intersection(cols_to_keep_from_tract_shapefile)].copy()
    
    # Rename columns
    census_tracts_gdf = census_tracts_gdf.rename(columns={
        'GEOID20': 'geo_id',
        'NAME20': 'tract_name'
    })
    print("Census Tracts columns standardized and renamed.")

    print("\n--- Census Tract Boundaries Loaded and Standardized ---")
    print("Cleaned Census Tract Boundaries GeoDataFrame Info:")
    print(census_tracts_gdf.info())
    print("\nCleaned Census Tract Boundaries Head:")
    print(census_tracts_gdf.head())

except FileNotFoundError:
    print(f"Error: Shapefile not found at {shp_file}. Please check the path.")
except Exception as e:
    print(f"An unexpected error occurred while reading or processing the shapefile: {e}")
if census_tracts_gdf is None or census_tracts_gdf.empty:
    print("Census Tract Boundaries GeoDataFrame is empty or was not loaded successfully.")



--- Creating census_tracts_gdf ---
Successfully loaded shapefile from: ../data/US_Census/Tract_Shapefiles/tl_2020_39049_tract20.shp
Initial Census Tract GeoDataFrame loaded. Total features: 328
Original CRS: EPSG:4269

Reprojecting Census Tract Boundaries from EPSG:4269 to EPSG:3735...
New Census Tract Boundaries CRS: EPSG:3735
Census Tracts columns standardized and renamed.

--- Census Tract Boundaries Loaded and Standardized ---
Cleaned Census Tract Boundaries GeoDataFrame Info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geo_id      328 non-null    object  
 1   tract_name  328 non-null    object  
 2   geometry    328 non-null    geometry
dtypes: geometry(1), object(2)
memory usage: 7.8+ KB
None

Cleaned Census Tract Boundaries Head:
        geo_id tract_name                                           geometry
0  39049000110   

In [34]:
print("\n--- Creating final_census_gdf ---")

if merged_census_df is not None and census_tracts_gdf is not None:
    final_census_gdf = merged_census_df.merge(census_tracts_gdf, on="geo_id", how="right")
    final_census_gdf = gpd.GeoDataFrame(
        final_census_gdf
    )
else:
    print("One or both Census DataFrames not loaded for merging.")
print("\n--- final_census_gdf created successfully! ---")


--- Creating final_census_gdf ---

--- final_census_gdf created successfully! ---


# Creating map

### Reproject gdfs, create color dictionary for COTA lines

In [35]:
# final_census_gdf exists from previous code
# cota_stops_gdf exists from previous code
# morpc_taz_cleaned_gdf exists from previous code

cota_lines_gdf = gpd.read_file("../data/COTA/COTA_Lines_Current.shp")

# reproject all gdfs
if final_census_gdf.crs != "EPSG:4326":
    final_census_gdf = final_census_gdf.to_crs(epsg=4326)
    print(f"Reprojected Census Tract GeoDataFrame to CRS: {final_census_gdf.crs}")
else:
    print("Census Tract GeoDataFrame is already in EPSG:4326.")
    
if cota_stops_gdf.crs != "EPSG:4326":
    cota_stops_gdf = cota_stops_gdf.to_crs(crs=4326)
    print(f"Reprojected COTA Stops GeoDataFrame to CRS: {cota_stops_gdf.crs}")
else:
    print("COTA Stops GeoDataFrame is already in ESPG:4326.")
    
if cota_lines_gdf.crs != "EPSG:4326":
    cota_lines_gdf = cota_lines_gdf.to_crs(crs=4326)
    print(f"Reprojected GeoDataFrame to CRS: {cota_lines_gdf.crs}")
else:
    print("COTA Stops GeoDataFrame is already in ESPG:4326.")
    
unique_abbrs = cota_lines_gdf["LineAbbr"].astype(str).unique()
chosen_colors = [
    "crimson", "teal", "goldenrod", "slateblue", "tomato",
    "darkgreen", "sienna", "orchid", "dodgerblue", "coral",
    "mediumvioletred", "steelblue", "peru", "indigo", "salmon",
    "forestgreen", "deeppink", "cadetblue", "firebrick", "mediumseagreen",
    "slategray", "navy", "mediumorchid", "darkorange", "mediumblue",
    "seagreen", "darkcyan", "chocolate", "mediumturquoise", "rosybrown",
    "indianred", "olivedrab", "darkslateblue", "limegreen", "plum",
    "orangered", "mediumslateblue", "royalblue", "turquoise"
]
abbr_to_color = {
    abbr: color for abbr, color in zip(unique_abbrs, chosen_colors)
}

Reprojected Census Tract GeoDataFrame to CRS: EPSG:4326
Reprojected COTA Stops GeoDataFrame to CRS: EPSG:4326
Reprojected GeoDataFrame to CRS: EPSG:4326


### Create Folium map

In [ ]:
center_lat = final_census_gdf.geometry.centroid.y.mean()
center_lon = final_census_gdf.geometry.centroid.x.mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=11)

# Add the GeoDataFrames as GeoJson layers to the Folium map

# --- Franklin County Tracts ---
tractTooltip = folium.GeoJsonTooltip(
    fields=["tract_name"],
    aliases=["Tract:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    final_census_gdf,
    name='Franklin County Tracts',
    style_function=lambda x: {
        'fillColor': '#43a2ca',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=tractTooltip,
    show=False
).add_to(m)

# --- TAZ Forecast ---
forecast2030Tooltip = folium.GeoJsonTooltip(
    fields=["jobs_2030_density_category"],
    aliases=["2030 Job Density Category: "],
    localize=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

forecast2040Tooltip = folium.GeoJsonTooltip(
    fields=["jobs_2040_density_category"],
    aliases=["2040 Job Density Category: "],
    localize=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

forecast2050Tooltip = folium.GeoJsonTooltip(
    fields=["jobs_2050_density_category"],
    aliases=["2050 Job Density Category: "],
    localize=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    morpc_taz_cleaned_gdf,
    name='TAZ 2030 Forecast',
    style_function=lambda x: {
        "fillColor": (
            "#AEEFFF" if x["properties"]["jobs_2040_density_category"] == "Lowest" else
            "#8DB9F1" if x["properties"]["jobs_2040_density_category"] == "Low" else
            "#A178E3" if x["properties"]["jobs_2040_density_category"] == "Medium" else
            "#9244D9" if x["properties"]["jobs_2040_density_category"] == "High" else
            "#7D1FBF" if x["properties"]["jobs_2040_density_category"] == "Highest" else
            "gray"
        ),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=forecast2030Tooltip,
    show=False
).add_to(m)

folium.GeoJson(
    morpc_taz_cleaned_gdf,
    name='TAZ 2040 Forecast',
    style_function=lambda x: {
        "fillColor": (
            "#AEEFFF" if x["properties"]["jobs_2040_density_category"] == "Lowest" else
            "#8DB9F1" if x["properties"]["jobs_2040_density_category"] == "Low" else
            "#A178E3" if x["properties"]["jobs_2040_density_category"] == "Medium" else
            "#9244D9" if x["properties"]["jobs_2040_density_category"] == "High" else
            "#7D1FBF" if x["properties"]["jobs_2040_density_category"] == "Highest" else
            "gray"
        ),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=forecast2040Tooltip,
    show=False
).add_to(m)

folium.GeoJson(
    morpc_taz_cleaned_gdf,
    name='TAZ 2050 Forecast',
    style_function=lambda x: {
        "fillColor": (
            "#AEEFFF" if x["properties"]["jobs_2050_density_category"] == "Lowest" else
            "#8DB9F1" if x["properties"]["jobs_2050_density_category"] == "Low" else
            "#A178E3" if x["properties"]["jobs_2050_density_category"] == "Medium" else
            "#9244D9" if x["properties"]["jobs_2050_density_category"] == "High" else
            "#7D1FBF" if x["properties"]["jobs_2050_density_category"] == "Highest" else
            "gray"
        ),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=forecast2050Tooltip,
    show=False
).add_to(m)

# --- COTA Lines ---
cota_lines_gdf["Length"] = cota_lines_gdf["Length"].astype(float).round(1)

cotaLinesTooltip = folium.GeoJsonTooltip(
    fields=["LineName", "Line_Label", "Length"],
    aliases=["Line Name:", "#:", "Length (miles):"],
    localize=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

cotaLinesVar = folium.GeoJson(
    cota_lines_gdf,
    name="COTA Lines",
    style_function=lambda x: {
        "weight": 10,
        "color": abbr_to_color[str(x["properties"]["LineAbbr"])],
        'opacity': 0.5,
        'background-color': abbr_to_color[str(x["properties"]["LineAbbr"])],
    },
    tooltip = cotaLinesTooltip,
).add_to(m)


# --- COTA Stops --- 
stopColorMap = cm.LinearColormap(["#0F0E0E", "#22A338"], vmin=0.0, vmax=63)

cotaStopsVar = folium.GeoJson(
    cota_stops_gdf,
    name="COTA Bus Stops",
    marker=folium.Circle(radius=10, fill_color="orange", fill_opacity=0.4, color="black", weight=1),
    popup=folium.GeoJsonPopup(
        fields=["stop_name", "lines_served", "total_onboards"],
        aliases=["Stop Name:", "Lines Served:", "Avg. Daily onboards:"]),
    style_function=lambda x: {
        "fillColor": stopColorMap(x["properties"]["total_onboards"]),
        "radius": 100,
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=False,
).add_to(m)

# --- Census Layers ---
# Households with no Vehicles
final_census_gdf["households_no_vehicle"] = (final_census_gdf["households_no_vehicle"].astype(float) / final_census_gdf["households_total"].astype(float)) * 100
final_census_gdf["households_no_vehicle"] = final_census_gdf["households_no_vehicle"].round(1)
householdsNoVehiclesColormap = cm.LinearColormap(["green", "yellow", "red"], vmin=0.0, vmax=final_census_gdf["households_no_vehicle"].max())
householdsNoVehiclesTooltip = folium.GeoJsonTooltip(
    fields=["households_no_vehicle"],
    aliases=["% of Households w/ no vehicles: "],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

final_census_gdf.fillna(0, inplace=True)
folium.GeoJson(
    final_census_gdf,
    name='Households w/ no Vehicles',
    style_function=lambda x: {
        "fillColor": householdsNoVehiclesColormap((x["properties"]['households_no_vehicle'])),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=householdsNoVehiclesTooltip,
    show=False
).add_to(m)

# % Commutes w/ Public Transit
final_census_gdf["commute_public_transit_total"] = (final_census_gdf["commute_public_transit_total"].astype(float) / final_census_gdf["workers_earnings_total"].astype(float)) * 100
final_census_gdf["commute_public_transit_total"] = final_census_gdf["commute_public_transit_total"].round(1)
percentPublicTransitColormap = cm.LinearColormap(["#ABE4BA", "#134A17"], vmin=0.0, vmax=final_census_gdf["commute_public_transit_total"].max())
percentPublicTransitTooltip = folium.GeoJsonTooltip(
    fields=["commute_public_transit_total"],
    aliases=["% of workers commuting via public transit:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

final_census_gdf.fillna(0, inplace=True)
folium.GeoJson(
    final_census_gdf,
    name='Workers commuting via public transit',
    style_function=lambda x: {
        "fillColor": percentPublicTransitColormap((x["properties"]['households_no_vehicle'])),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=percentPublicTransitTooltip,
    show=False
).add_to(m)

# Workers commuting alone via car
final_census_gdf["commute_drove_alone_total"] = (final_census_gdf["commute_drove_alone_total"].astype(float) / final_census_gdf["workers_earnings_total"].astype(float)) * 100
final_census_gdf["commute_drove_alone_total"] = final_census_gdf["commute_drove_alone_total"].round(1)
percentCommuteAloneColormap = cm.LinearColormap(["blue", "purple", "red"], vmin=45, vmax=final_census_gdf["commute_drove_alone_total"].max())
percentCommuteAloneTooltip = folium.GeoJsonTooltip(
    fields=["commute_drove_alone_total"],
    aliases=["% of workers commuting alone via car:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

final_census_gdf.fillna(0, inplace=True)
folium.GeoJson(
    final_census_gdf,
    name='Workers commuting alone via car',
    style_function=lambda x: {
        "fillColor": percentCommuteAloneColormap((x["properties"]['commute_drove_alone_total'])),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=percentCommuteAloneTooltip,
    show=False
).add_to(m)

# % Workers living under poverty line
final_census_gdf["workers_poverty_below_100"] = (final_census_gdf["workers_poverty_below_100"].astype(float) / final_census_gdf["workers_poverty_total"].astype(float)) * 100
final_census_gdf["workers_poverty_below_100"] = final_census_gdf["workers_poverty_below_100"].round(1)
percentPovertyColormap = cm.LinearColormap(["yellow", "purple"], vmin=final_census_gdf["workers_poverty_below_100"].min(), vmax=final_census_gdf["workers_poverty_below_100"].max())
percentPovertyTooltip = folium.GeoJsonTooltip(
    fields=["workers_poverty_below_100"],
    aliases=["% of workers living under poverty line: "],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

final_census_gdf.fillna(0, inplace=True)
folium.GeoJson(
    final_census_gdf,
    name='Workers under poverty line',
    style_function=lambda x: {
        "fillColor": percentPovertyColormap((x["properties"]['workers_poverty_below_100'])),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.7
    },
    tooltip=percentPovertyTooltip,
    show=False
).add_to(m)

# Keeps COTA Stops and Lines layers in front no matter what variables are activated
m.keep_in_front(cotaLinesVar)
m.keep_in_front(cotaStopsVar)

# Add a layer control to toggle layers
folium.LayerControl().add_to(m)

# Save the map to an HTML file
map_output_path = "../map.html"
m.save(map_output_path)
print(f"\nMap with shapefile saved to {map_output_path}")

C:\Users\erolz\AppData\Local\Temp\ipykernel_31160\1099266861.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = final_census_gdf.geometry.centroid.y.mean()
C:\Users\erolz\AppData\Local\Temp\ipykernel_31160\1099266861.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = final_census_gdf.geometry.centroid.x.mean()



Map with shapefile saved to ../map.html
